# Inspecting crop phenology

* **Products used:** 
[ndvi_anomaly](https://explorer.digitalearth.africa/ndvi_anomaly)

## Background
Show the relationships between visual inspection of NDVI across space, mean NDVI through time, and NDVI anomalies.

## Description
Animation
***

## Getting started

**To run this analysis**, run all the cells in the notebook, starting with the "Load packages and apps" cell.

### Load packages and apps
Description of anim package and code

In [1]:
%matplotlib inline

import datacube
import datetime as dt
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import matplotlib.dates as mdates

In [2]:
dc = datacube.Datacube(app='Cropphenology')

### Analysis parameters

---


In [3]:
# Define the area of interest for the analysis
lat = -21.05
lon = 31.56
buffer = 0.05
year = "2021"

In [4]:
latitude = (lat - buffer, lat + buffer)
longitude = (lon - buffer, lon + buffer)

In [5]:
start_date = dt.datetime.strptime(year, "%Y").replace(month=1, day=1)
end_date = dt.datetime.strptime(year, "%Y").replace(month=12, day=31)

In [6]:
# Construct the data cube query    
query = {
       'x': longitude,
       'y': latitude,
       'time': (start_date, end_date),
       'output_crs': 'EPSG:6933',
    'resolution': (-30, 30)
    }

    # Load the data and mask out bad quality pixels
ds = dc.load(product="ndvi_anomaly", **query)
ds

<xarray.Dataset>
Dimensions:           (time: 12, y: 398, x: 323)
Coordinates:
  * time              (time) datetime64[ns] 2021-01-16T11:59:59.999999 ... 20...
  * y                 (y) float64 -2.621e+06 -2.621e+06 ... -2.633e+06
  * x                 (x) float64 3.04e+06 3.04e+06 ... 3.05e+06 3.05e+06
    spatial_ref       int32 6933
Data variables:
    ndvi_mean         (time, y, x) float32 0.6586 0.5823 ... 0.7492 0.7441
    ndvi_std_anomaly  (time, y, x) float32 -0.1129 -0.7352 ... 0.8299 0.7802
    clear_count       (time, y, x) int8 8 8 8 8 7 7 7 7 7 ... 7 7 7 7 7 7 7 7 7
Attributes:
    crs:           EPSG:6933
    grid_mapping:  spatial_ref

In [7]:
masked_ds = ds.ndvi_mean

masked_ds_mean = masked_ds.mean(dim=['x', 'y'], skipna=True)

# create a figure and axes
fig = plt.figure(figsize=(10,8))
ax1 = plt.subplot(211)   
ax2 = plt.subplot(223)


cax = ds.ndvi_mean[0,:,:].plot(
    add_colorbar=True,
    cmap='RdYlGn',
    vmin=-1, vmax=1,
    cbar_kwargs={
        'extend':'neither'
    },
    ax=ax2
)


ax1.set_ylim([0, 1])
ax1.set_title("Mean NDVI")
ax1.set_xlabel("Date")
ax1.set_ylabel("NDVI")

ax3 = plt.subplot(224)

ndvi_anom = ds.ndvi_std_anomaly.mean(dim=['x', 'y']).to_dataframe().drop(['spatial_ref'], axis=1)

ax3.clear()
ax3.set_ylim([-4, 4])
ax3.xaxis_date()
ax3.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
ax3.set_title("NDVI Anomaly")
ax3.set_xlabel("Date")
ax3.set_ylabel("Std Deviations from monthly mean")


def update(num, x, y, line):
    y2[num] =  ndvi_anom.ndvi_std_anomaly[num]
    bars[num].set_height(y2[num])
    cax.set_array(ds.ndvi_mean[(num),:,:].values.flatten())
    ax2.set_title("Time = " + str(ds.ndvi_mean.coords['time'].values[(num)])[:12])
    line.set_data(x[:num+1], y[:num+1])
    return line,
    
x = masked_ds_mean.time
y = masked_ds_mean
                                                                         
x2 = ndvi_anom.index
y2 = [0] * ndvi_anom.ndvi_std_anomaly
bars = ax3.bar(x2, y2, align='center', width=20,color=(ndvi_anom.ndvi_std_anomaly > 0).map({True: 'g', False: 'brown'}))

line, = ax1.plot(x, y, marker='*',
                color='blue')
plt.tight_layout()
ani = animation.FuncAnimation(fig, update, len(x), fargs=[x, y, line],
                              interval=400, blit=True)

plt.close()
HTML(ani.to_html5_video())

## Drawing conclusions

Interpret each product...

---

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Compatible datacube version:**

In [8]:
print(datacube.__version__)

1.8.8


**Last Tested:**

In [9]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')

'2023-03-27'